<a href="https://colab.research.google.com/github/PosgradoMNA/actividades-del-projecto-equipo-125/blob/main/Reto_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Reto, 2da entrega
Estefania Abigail Castro Belmont A01332636

Vladimir Salazar Altamirano A01793118

Materia: Ciencia y analítica de datos

Profesor: Maria de la Paz Rico

Tutor: Julio César Galindo López

18/11/2022

Clasificación
En esta base de datos encontraras:

**Aguas subterráneas**

Utilizando la base de datos que hayas elegido y hayas realizado su limpieza en la anterior entrega del reto, realiza lo siguiente:

Selecciona tus variables independientes X y dependiente Y (semáforo)

Cambia a label encoding el semáforo, ej, de ["clase 1", "clase 2", "clase 3"] a [ 1,2,3]

Realiza un análisis general de las features importances a traves de decision trees o random forest.

Selecciona las variables de mayor importancia.

Realiza tu clasificador, recuerda dividir los datos de manera balanceada (auxiliate de train test split)

Explora que clasificador es el más optimo, ejemplo:
Decision trees

Random Forest.

Determina el grado de exactitud a través del reporte de clasificación y análisis de la gráfica de Precision Recall.

Visualiza los resultados del modelo o las predicciones a través de una matriz de confusión.

Realiza un reporte de los principales hallazgos y conclusiones del experimento e incluyelos en una presentación ejecutiva de 10 diapositivas. La presentación deberá incluir todos los pasos del pipeline seguidos, limpieza, análisis, kmeans, clasificación, resultados y conclusiones.

##Base de datos de aguas subterraneas

In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/PosgradoMNA/actividades-del-projecto-equipo-125/main/Datos_de_calidad_del_agua_de_sitios_de_monitoreo_de_aguas_subterraneas_2020.csv',
                 sep=',',
                 encoding='cp1252')
                 
df.head()

,CLAVE,SITIO,ORGANISMO_DE_CUENCA,ESTADO,MUNICIPIO,ACUIFERO,SUBTIPO,LONGITUD,LATITUD,PERIODO,...,CUMPLE_CON_DUR,CUMPLE_CON_CF,CUMPLE_CON_NO3,CUMPLE_CON_AS,CUMPLE_CON_CD,CUMPLE_CON_CR,CUMPLE_CON_HG,CUMPLE_CON_PB,CUMPLE_CON_MN,CUMPLE_CON_FE
0,DLAGU6,POZO SAN GIL,LERMA SANTIAGO PACIFICO,AGUASCALIENTES,ASIENTOS,VALLE DE CHICALOTE,POZO,-102.02210,22.20887,2020,...,SI,SI,SI,SI,SI,SI,SI,SI,SI,SI
1,DLAGU6516,POZO R013 CAÑADA HONDA,LERMA SANTIAGO PACIFICO,AGUASCALIENTES,AGUASCALIENTES,VALLE DE CHICALOTE,POZO,-102.20075,21.99958,2020,...,SI,SI,SI,SI,SI,SI,SI,SI,SI,SI
2,DLAGU7,POZO COSIO,LERMA SANTIAGO PACIFICO,AGUASCALIENTES,COSIO,VALLE DE AGUASCALIENTES,POZO,-102.28801,22.36685,2020,...,SI,SI,SI,NO,SI,SI,SI,SI,SI,SI
3,DLAGU9,POZO EL SALITRILLO,LERMA SANTIAGO PACIFICO,AGUASCALIENTES,RINCON DE ROMOS,VALLE DE AGUASCALIENTES,POZO,-102.29449,22.18435,2020,...,SI,SI,SI,SI,SI,SI,SI,SI,SI,SI
4,DLBAJ107,RANCHO EL TECOLOTE,PENINSULA DE BAJA CALIFORNIA,BAJA CALIFORNIA SUR,LA PAZ,TODOS SANTOS,POZO,-110.24480,23.45138,2020,...,SI,SI,NO,SI,SI,SI,SI,SI,SI,SI


##Limpieza anterior

In [3]:
ndf = df.copy()
ndf.drop(columns=['SDT_mg/L','CONTAMINANTES'], inplace=True)

In [4]:
num_posibles=["SDT_M_mg/L","FLUORUROS_mg/L","DUR_mg/L","COLI_FEC_NMP/100_mL",
              "N_NO3_mg/L","AS_TOT_mg/L","CD_TOT_mg/L","CR_TOT_mg/L",
              "HG_TOT_mg/L","PB_TOT_mg/L","MN_TOT_mg/L","FE_TOT_mg/L"]

ndf[num_posibles]=ndf[num_posibles].replace({'<':''}, regex=True)
ndf[num_posibles]=ndf[num_posibles].astype(float)

In [5]:
cual_col=[]
num_col=[]
for n in ndf:
  if (ndf[n].dtype)=='object':
    cual_col.append(n)
  else:
    num_col.append(n)

In [6]:
for c in num_col:
  ndf[c]=ndf[c].fillna(ndf[c].median())
for c in cual_col:
  ndf[c]=ndf[c].fillna(ndf[c].mode()[0])

In [7]:
ndf.isnull().sum()

CLAVE                   0
SITIO                   0
ORGANISMO_DE_CUENCA     0
ESTADO                  0
MUNICIPIO               0
ACUIFERO                0
SUBTIPO                 0
LONGITUD                0
LATITUD                 0
PERIODO                 0
ALC_mg/L                0
CALIDAD_ALC             0
CONDUCT_mS/cm           0
CALIDAD_CONDUC          0
SDT_M_mg/L              0
CALIDAD_SDT_ra          0
CALIDAD_SDT_salin       0
FLUORUROS_mg/L          0
CALIDAD_FLUO            0
DUR_mg/L                0
CALIDAD_DUR             0
COLI_FEC_NMP/100_mL     0
CALIDAD_COLI_FEC        0
N_NO3_mg/L              0
CALIDAD_N_NO3           0
AS_TOT_mg/L             0
CALIDAD_AS              0
CD_TOT_mg/L             0
CALIDAD_CD              0
CR_TOT_mg/L             0
CALIDAD_CR              0
HG_TOT_mg/L             0
CALIDAD_HG              0
PB_TOT_mg/L             0
CALIDAD_PB              0
MN_TOT_mg/L             0
CALIDAD_MN              0
FE_TOT_mg/L             0
CALIDAD_FE  

##Selecciona tus variables independientes X y dependiente Y (semáforo)

In [8]:
X=ndf[['CLAVE', 'SITIO', 'ORGANISMO_DE_CUENCA', 'ESTADO', 'MUNICIPIO',
       'ACUIFERO', 'SUBTIPO', 'LONGITUD', 'LATITUD', 'PERIODO', 'ALC_mg/L',
       'CALIDAD_ALC', 'CONDUCT_mS/cm', 'CALIDAD_CONDUC', 'SDT_M_mg/L',
       'CALIDAD_SDT_ra', 'CALIDAD_SDT_salin', 'FLUORUROS_mg/L', 'CALIDAD_FLUO',
       'DUR_mg/L', 'CALIDAD_DUR', 'COLI_FEC_NMP/100_mL', 'CALIDAD_COLI_FEC',
       'N_NO3_mg/L', 'CALIDAD_N_NO3', 'AS_TOT_mg/L', 'CALIDAD_AS',
       'CD_TOT_mg/L', 'CALIDAD_CD', 'CR_TOT_mg/L', 'CALIDAD_CR', 'HG_TOT_mg/L',
       'CALIDAD_HG', 'PB_TOT_mg/L', 'CALIDAD_PB', 'MN_TOT_mg/L', 'CALIDAD_MN',
       'FE_TOT_mg/L', 'CALIDAD_FE', 'CUMPLE_CON_ALC',
       'CUMPLE_CON_COND', 'CUMPLE_CON_SDT_ra', 'CUMPLE_CON_SDT_salin',
       'CUMPLE_CON_FLUO', 'CUMPLE_CON_DUR', 'CUMPLE_CON_CF', 'CUMPLE_CON_NO3',
       'CUMPLE_CON_AS', 'CUMPLE_CON_CD', 'CUMPLE_CON_CR', 'CUMPLE_CON_HG',
       'CUMPLE_CON_PB', 'CUMPLE_CON_MN', 'CUMPLE_CON_FE']]
y=ndf[['SEMAFORO']]

##Cambia a label encoding el semáforo, ej, de ["clase 1", "clase 2", "clase 3"] a [ 1,2,3]

In [9]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [10]:
y['SEMAFORO']=le.fit_transform(y['SEMAFORO'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [11]:
le.classes_

array(['Amarillo', 'Rojo', 'Verde'], dtype=object)

In [12]:
le.inverse_transform([0,1,2])

array(['Amarillo', 'Rojo', 'Verde'], dtype=object)

In [13]:
for c in X.columns:
  X[c]=le.fit_transform(X[c])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


##Realiza un análisis general de las features importances a traves de decision trees o random forest.

In [14]:
rnd_clf = RandomForestClassifier(n_estimators=500, random_state=42)
rnd_clf.fit(X, y)
for score, name in zip(rnd_clf.feature_importances_, X.columns):
  print(round(score, 2),name)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


0.01 CLAVE
0.0 SITIO
0.0 ORGANISMO_DE_CUENCA
0.0 ESTADO
0.01 MUNICIPIO
0.0 ACUIFERO
0.0 SUBTIPO
0.01 LONGITUD
0.01 LATITUD
0.0 PERIODO
0.01 ALC_mg/L
0.0 CALIDAD_ALC
0.03 CONDUCT_mS/cm
0.0 CALIDAD_CONDUC
0.03 SDT_M_mg/L
0.01 CALIDAD_SDT_ra
0.01 CALIDAD_SDT_salin
0.08 FLUORUROS_mg/L
0.07 CALIDAD_FLUO
0.05 DUR_mg/L
0.05 CALIDAD_DUR
0.04 COLI_FEC_NMP/100_mL
0.01 CALIDAD_COLI_FEC
0.04 N_NO3_mg/L
0.04 CALIDAD_N_NO3
0.05 AS_TOT_mg/L
0.01 CALIDAD_AS
0.0 CD_TOT_mg/L
0.0 CALIDAD_CD
0.01 CR_TOT_mg/L
0.01 CALIDAD_CR
0.0 HG_TOT_mg/L
0.0 CALIDAD_HG
0.01 PB_TOT_mg/L
0.0 CALIDAD_PB
0.03 MN_TOT_mg/L
0.02 CALIDAD_MN
0.03 FE_TOT_mg/L
0.02 CALIDAD_FE
0.01 CUMPLE_CON_ALC
0.01 CUMPLE_CON_COND
0.0 CUMPLE_CON_SDT_ra
0.0 CUMPLE_CON_SDT_salin
0.07 CUMPLE_CON_FLUO
0.04 CUMPLE_CON_DUR
0.04 CUMPLE_CON_CF
0.03 CUMPLE_CON_NO3
0.04 CUMPLE_CON_AS
0.0 CUMPLE_CON_CD
0.01 CUMPLE_CON_CR
0.0 CUMPLE_CON_HG
0.0 CUMPLE_CON_PB
0.02 CUMPLE_CON_MN
0.02 CUMPLE_CON_FE


##Selecciona las variables de mayor importancia.

In [15]:
#0.01 CLAVE
#0.01 MUNICIPIO
#0.01 LONGITUD
#0.01 LATITUD
#0.01 ALC_mg/L
#0.01 CALIDAD_SDT_ra
#0.01 CALIDAD_SDT_salin
#0.01 CALIDAD_COLI_FEC
#0.01 CALIDAD_AS
#0.01 CR_TOT_mg/L
#0.01 CALIDAD_CR
#0.01 PB_TOT_mg/L
#0.01 CUMPLE_CON_ALC
#0.01 CUMPLE_CON_COND
#0.01 CUMPLE_CON_CR

#0.03 CONDUCT_mS/cm
#0.03 SDT_M_mg/L
#0.08 FLUORUROS_mg/L
#0.07 CALIDAD_FLUO
#0.05 DUR_mg/L
#0.05 CALIDAD_DUR
#0.04 COLI_FEC_NMP/100_mL
#0.04 N_NO3_mg/L
#0.04 CALIDAD_N_NO3
#0.05 AS_TOT_mg/L
#0.03 MN_TOT_mg/L
#0.02 CALIDAD_MN
#0.03 FE_TOT_mg/L
#0.02 CALIDAD_FE
#0.07 CUMPLE_CON_FLUO
#0.04 CUMPLE_CON_DUR
#0.04 CUMPLE_CON_CF
#0.03 CUMPLE_CON_NO3
#0.04 CUMPLE_CON_AS
#0.02 CUMPLE_CON_MN
#0.02 CUMPLE_CON_FE


In [16]:
X = ndf[['CONDUCT_mS/cm','SDT_M_mg/L','FLUORUROS_mg/L','CALIDAD_FLUO','DUR_mg/L',
        'CALIDAD_DUR','COLI_FEC_NMP/100_mL','N_NO3_mg/L','CALIDAD_N_NO3','AS_TOT_mg/L',
        'MN_TOT_mg/L','CALIDAD_MN','FE_TOT_mg/L','CALIDAD_FE','CUMPLE_CON_FLUO',
        'CUMPLE_CON_DUR','CUMPLE_CON_CF','CUMPLE_CON_NO3','CUMPLE_CON_AS','CUMPLE_CON_MN',
        'CUMPLE_CON_FE']]

In [18]:
for c in X.columns:
  X[c]=le.fit_transform(X[c])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


##Realiza tu clasificador, recuerda dividir los datos de manera balanceada (auxiliate de train test split)

In [28]:
from sklearn.tree import DecisionTreeClassifier

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, stratify=y, random_state=42)

voting_clf = VotingClassifier(
    estimators=[
        ('lr', LogisticRegression(random_state=42)),
        ('rf', RandomForestClassifier(random_state=42)),
        ('svc', SVC(max_iter=100000, random_state=42)),
        ('dt', DecisionTreeClassifier( random_state=42))
    ]
)
voting_clf.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regressio

VotingClassifier(estimators=[('lr', LogisticRegression(random_state=42)),
                             ('rf', RandomForestClassifier(random_state=42)),
                             ('svc', SVC(max_iter=100000, random_state=42)),
                             ('dt', DecisionTreeClassifier(random_state=42))])

##Explora que clasificador es el más optimo, ejemplo: Decision trees, Random Forest.

In [29]:
for name, clf in voting_clf.named_estimators_.items():
    print(name, "=", clf.score(X_test, y_test))

lr = 0.6495327102803738
rf = 0.9579439252336449
svc = 0.8037383177570093
dt = 0.9112149532710281




Determina el grado de exactitud a través del reporte de clasificación y análisis de la gráfica de Precision Recall.

Visualiza los resultados del modelo o las predicciones a través de una matriz de confusión.

Realiza un reporte de los principales hallazgos y conclusiones del experimento e incluyelos en una presentación ejecutiva de 10 diapositivas. La presentación deberá incluir todos los pasos del pipeline seguidos, limpieza, análisis, kmeans, clasificación, resultados y conclusiones.